# Proving the Playpen Agent/CodePlan for Compiler Errors

We need prove that the playpen code plan agents workflow will work as expected. This will be a way to run them and make sure that things work as we expect.

## Setup

In [4]:
import pprint

pp = pprint.PrettyPrinter(indent=2)

## Create the Test Environment

For creating the test environment we will:

1. move the coolstore to a temp direcotry
2. Update the InventoryEntity to use UUID for the itemID as the type.

In [5]:
import tempfile
import shutil
import os
import subprocess

#First we need to download coolstore if it does not exist.

if not os.path.exists("./coolstore"):
    subprocess.run("../../example/fetch.sh")

if not os.path.exists("../../kai-analyzer/kai-analyzer"):
    subprocess.run(["go", "build", "-o", "kai-analyzer", "main.go"], cwd="../../kai-analyzer")

temp_dir = tempfile.TemporaryDirectory()
coolstore_path = os.path.join(temp_dir.name, "coolstore")
shutil.copytree("./coolstore", coolstore_path)

## make the necessary change

print(temp_dir)


Cloning into 'coolstore'...


<TemporaryDirectory '/var/folders/vt/5bfp7vyd1h79_7k5ygr0fttr0000gn/T/tmpowrnp22w'>


## Set up the Code Plan types and run code plan.

In [7]:
from pathlib import Path
from playpen.repo_level_awareness.api import RpcClientConfig
from playpen.repo_level_awareness.codeplan import TaskManager
from playpen.repo_level_awareness.task_runner.analyzer_lsp.validator import AnlayzerLSPStep
from playpen.repo_level_awareness.task_runner.analyzer_lsp.task_runner import AnalyzerTaskRunner
from kai.service.llm_interfacing.model_provider import ModelProvider
from kai.models.kai_config import KaiConfig
from playpen.repo_level_awareness.vfs.git_vfs import RepoContextManager


config = RpcClientConfig(Path(coolstore_path),
                         "../../kai-analyzer/kai-analyzer",
                         "/Users/shurley/repos/MTA/rulesets/default/generated",
                         Path("/Users/shurley/repos/kai/jdtls/bin/jdtls"),
                         Path("./java-bundle/java-analyzer-bundle.core-1.0.0-SNAPSHOT.jar"),
                         "konveyor.io/target=quarkus",
                         None,
                         None)

kai_config = KaiConfig.model_validate_filepath("../../kai/config.toml")
modelProvider = ModelProvider(kai_config.models)
rcm = RepoContextManager(config.repo_directory, modelProvider.llm)


agent = AnalyzerTaskRunner(modelProvider.llm)

task_manager = TaskManager(
        config,
        rcm,
        None,
        validators=[AnlayzerLSPStep(config)],
        agents=[agent],
    )


for task in task_manager.get_next_task():
    pp.pprint(task)
    task_manager.supply_result(task_manager.execute_task(task))

TypeError: RpcClientConfig.__init__() takes 7 positional arguments but 9 were given

## Cleanup temp dir

In [10]:
temp_dir.cleanup()